# Mapping Web Case Study

## Requiremennts

***Please  run only once***

In [ ]:
!pip install --upgrade ipywidgets
!pip install --upgrade pandas
!pip install --upgrade xlsxwriter
!pip install --upgrade mysql-connector-python
!pip install --upgrade numpy 

## Start

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML

import mysql.connector
import pandas as pd

In [ ]:
MAP_DATA_FILE_NAME = "map-data.csv"
RESULT_FILE_NAME = 'awo300.csv'

#### Connection info to prod database

*Note: user should enabled external connections*

In [ ]:
HOST = 'smart-space.co.uk'
DATABASE = 'Please fill'
USER = 'Please fill'
PASSWORD = 'Please fill'

#### Read case studies information

In [ ]:
case_study_df = []

In [ ]:
try:
    
    connection = mysql.connector.connect(
        host = HOST,
        user = USER,
        password = PASSWORD,
        database = DATABASE 
    )

except mysql.connector.Error as err:
    display(Javascript("alert('Failed to connect to MySQL database, please check credentials above')"))

if connection.is_connected():
    cursor = connection.cursor(dictionary=True)

    try:
        query = """
        SELECT 
            B.`id`, 
            B.`longtitle` as case_study,
            B.`uri` as case_study_uri, 
            `value`, 
            C.`longtitle` as testinomials, 
            C.`uri` as testinomials_uri 
        FROM 
            `modx_site_tmplvar_contentvalues` as A 
            INNER JOIN `modx_site_content` as B ON A.`contentid` = B.`id` 
            INNER JOIN `modx_site_content` as C ON A.`value` = C.`id` 
        WHERE 
            `tmplvarid` = 163"""
        
        cursor.execute(query)

        rows = cursor.fetchall()
        for row in rows:
            case_study_df.append(row)

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        cursor.close()
        connection.close()

else:
    display(Javascript("alert('Failed to read from MySQL database, please check permissions')"))

In [ ]:
case_study_df = pd.DataFrame(case_study_df)
case_study_df.head()

#### Upload map data

In [ ]:
upload_button = widgets.FileUpload(description="Upload map data in csv format, with separator';'", accept=".csv")

def handle_upload(change):
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_content = upload_button.value[uploaded_filename]["content"]

    with open(MAP_DATA_FILE_NAME, "wb") as f:
        f.write(uploaded_content)
    print(f"File '{uploaded_filename}' uploaded successfully.")

upload_button.observe(handle_upload, names="value")

display(upload_button)
upload_button._dom_classes = ("upload-button",)

Javascript("""
    let upload_buttons = document.querySelectorAll('.upload-button');
    upload_buttons.forEach((element) => element.click());
""")

In [ ]:
map_data_df = pd.read_csv(MAP_DATA_FILE_NAME, sep=";").fillna("")

map_data_df["Web Title Testimonials"] = ""
map_data_df["Web Testimonials"] = ""

map_data_df.head()

In [ ]:
df1 = pd.read_csv("PVG Map Data 23-11-01.csv", sep=";").fillna("")
df1.head()

#### Join data

In [ ]:
BASE_URL = "https://smart-space.co.uk/"
for i in range(len(map_data_df)):
    map_data_cs_url = map_data_df['Web Case Study'][i]
    map_data_cs_url = map_data_cs_url.replace(BASE_URL,"").strip('/')
    
    if len(map_data_cs_url) == 0:
        continue
    
    for j in range(len(case_study_df)):
        case_study_url = case_study_df['case_study_uri'][j]
        case_study_url = case_study_url.strip('/')
        
        if not(map_data_cs_url == case_study_url):
            continue
        
        name = map_data_df['NAME'][i]
        print(name)
        print(BASE_URL + map_data_cs_url)
        map_data_df['Web Title Testimonials'][i] = case_study_df['testinomials'][j]
        map_data_df['Web Testimonials'][i] = BASE_URL + case_study_df['testinomials_uri'][j]

In [ ]:
map_data_df.drop(map_data_df[map_data_df["Installation Address Postcode"] == ""].index, inplace=True)

for column in ["Building Type", "Hire Or Buy", "Application"]:
    map_data_df[column] = map_data_df[column].str.capitalize()

In [ ]:


map_data_df.to_csv(RESULT_FILE_NAME, sep = ';', index=False)
map_data_df.head()

In [ ]:
download_link = "<h3>Please convert file from link bellow to xlsx format and upload to folder data as awo300.xlsx</h3>"
download_link += f'<a href="{RESULT_FILE_NAME}" download>Download File ({RESULT_FILE_NAME})</a>'

HTML(download_link)